We will perform node classification on the standard **Cora** benchmark.


*   Citation network: classify papers into topics
*   2708 different papers (nodes), 5429 edges
*   Features correspond to bad of words that represent the paper. 7 topics
*   140 training, 500 validation, 1000 test
*   We will implement **Sum Pooling**, **Mean Pooling**, and **GCN**



In [ ]:
import numpy
import tensorflow as tf
!pip install spektral

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.1/140.1 kB 4.0 MB/s eta 0:00:00


In [ ]:
import spektral
import numpy as np

Spektral has convenience functions that allow us to quickly load and process standard Graph representation such as Cora.

In [ ]:
adj, features, labels, train_mask, val_mask, test_mask = spektral.datasets.citation.load_data(dataset_name='cora')

# convert sparse to dense represenation
features = features.todense()
adj = adj.todense() + np.eye(adj.shape[0])
features = features.astype('float32')

print(features.shape)
print(adj.shape)
print(labels.shape)

print(np.sum(train_mask))
print(np.sum(val_mask))
print(np.sum(test_mask))


ValueError: ignored

In [ ]:
def masked_softmax_cross_entropy(logits,mask):
  loss = tf.nn.softmax_cross_entropy_with_logits(logits = logits, labels=labels)
  mask = tf.cast(mask, dtype=tf.float32)
  mask/= tf.reduce_mean(mask)
  loss *= mask
  return tf.reduce_mean(loss)

def masked_accuracy(logits,labels,mask):
  correct_prediction = tf.equal(tf.argmax(logits,1),tf.argmax(labels,1))
  accuracy_all = tf.cast(correct_prediction, dtype=tf.float32)
  mask = tf.cast(mask, dtype=tf.float32)
  accuracy_all *= mask
  return tf.reduce_mean (accuracy_all)



In [ ]:
# define gnn
def gnn(node_feat,adj, transform,activation):
  seq_node_feat = transform(node_feat)
  feat_res = tf.matmul(adj,seq_node_feat)
  return activation(feat_res)


In [ ]:
# 2 layer gnn
def train_cora(node_feat,adj,gnn_fn,units,epochs,lr):
  lyr_1 = tf.keras.layers.Dense(units)
  lyr_2 = tf.keras.layers.Desne(7) # since there is only 7 classes

  def cora_gnn(node_feat,adj):
    hidden = gnn_fn(node_feat,adj,lyr_1,tf.nn.relu)
    logits = gnn_fn(hidden, adj, lyr_2, tf.identity)
    return logits

  optimizer = tf.keras.optimizers.Adam(learning_rate = lr)

  best_accuracy = 0.0
  for ep in range(epochs+1):
    with tf.GradientTape() as tape:
      logits = cora_gnn(node_feat,adj)
      loss = masked_softmax_cross_entropy(logits,labels,train_mask)

    variables = tape.watched_variables()
    grads = tape.gradient(loss,variables)
    optimizer.apply_gradients(zip(grads,variables))

    logits = cora_gnn(node_feat,adj)
    val_accuracy = masked_accuracy(logits,labels,val_mask)
    test_accuracy = masked_accuracy(logits,labels,test_mask)

    if val_accuracy > best_accuracy:
      best_accuracy = val_accuracy
      print('Epoch ',ep,'| Training loss:',loss.numpy(),'| val accuracy:',val_accuracy.numpy(),'| Test accuracy:',test_accuracy.numpy())


In [ ]:
train_cora(features,adj,gnn,32,200,0.01)

In [ ]:
train_cora(features,tf.eye(adj.shape[0]),gnn,32,200,0.01) #identity matrix

In [ ]:
deg = tf.reduce_sum(adj,axis=-1)
train_cora(features,adj/deg,gnn,32,200,0.01) #normalization by degree

In [ ]:
norm_deg = tf.linalg.diag(1.0/tf.sqrt(deg))
norm_adj = tf.matmul(norm_deg,tf.matmul(adj,norm_deg)) #GCN
train_cora(features,norm_adj,gnn,32,200,0.01)